## Recursive Summarizer
This code was drafted by GPT-4. 

In [1]:
import os
import dotenv
import nltk
import openai
from collections import deque

dotenv.load_dotenv(dotenv_path="./.env.local")
openai.api_key = os.environ["gpt_api_secret"]

def tokenize(text):
    return nltk.word_tokenize(text)

def detokenize(tokens):
    return ' '.join(tokens)

def summarize(text):
    prompt = f"{text}\n\nSummarize:"

    result = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        temperature=0.25,
        max_tokens=700
    )
    return result.choices[0].text.strip()


def recursive_summarization(text, section_size=3000, overlap=500):
    tokens = tokenize(text)
    summaries = []

    # Create overlapping sections
    sections = deque()
    start = 0
    while start < len(tokens):
        sections.append(tokens[start:start + section_size])
        start += section_size - overlap

    # Summarize each section and append to the summaries list
    while len(sections) > 1:
        section = sections.popleft()
        summary = summarize(detokenize(section))
        summaries.append(summary)

        # Break the summarized sections into smaller sections again
        if len(summaries) > 1:
            sections.append(tokenize(summaries.pop()) + tokenize(summaries.pop()))

        else:
            return summaries[0]
    


hi


## Non-recursive Summarizer
Instead of taking the summary of the summaries, the summaries are joined together. 
This technique created strange results in that they included repititive summaries and HTML headers. 
Therefore, I have commented it out only to keep track of the results based on this method. 

In [ ]:
# import os
# import dotenv
# import nltk
# import openai
# from collections import deque

# dotenv.load_dotenv(dotenv_path="./.env.local")
# openai.api_key = os.environ["gpt_api_secret"]

# def tokenize(text):
#     return nltk.word_tokenize(text)

# def detokenize(tokens):
#     return ' '.join(tokens)

# def summarize(text):
#     prompt = f"{text}\n\nSummarize:"

#     result = openai.Completion.create(
#         engine="text-davinci-002",
#         prompt=prompt,
#         temperature=0.1,
#         max_tokens=500
#     )
#     return result.choices[0].text.strip()

# def recursive_summarization(text, section_size=3000, overlap=500):
#     tokens = tokenize(text)
#     sections = deque()
#     start = 0
#     final_text = ""

#     while start < len(tokens):
#         sections.append(tokens[start:start + section_size])
#         start += section_size - overlap

#     # Summarize each section and directly append it to the final text
#     while len(sections) > 0:
#         section = sections.popleft()
#         summary = summarize(detokenize(section))
#         final_text += " " + summary
    
#     return final_text.strip()  # strip leading/trailing white space


In [2]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as pdf_file_obj:
        pdf_reader = PyPDF2.PdfReader(pdf_file_obj)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page_obj = pdf_reader.pages[page_num]
            text += page_obj.extract_text()
    return text

pdf_path = "/Users/seeker/Desktop/Axé Engineering/sample documents/APPLE_US_TERMS_COND-0056.pdf"
text = extract_text_from_pdf(pdf_path)
print(len(tokenize(text)))
print(text)


10738
 
OL-AMR -56 v. 2. 9 
 APPLE INC.  
PURCHASE AGREEMENT  
PURCHASE ORDER TERMS AND CONDITIONS  
THIS PURCHASE AGREEMENT (the "Agreement" ) sets forth the terms and conditions that apply to all purchases 
of goods and services by Apple from Seller by means of a purchase order (a "PO" ) issued by Apple to Seller. As used 
in this Agreement, "Seller"  means the entity identified on the face of a P O as "Seller" and its subsidiaries and 
affiliates, and "Apple"  means Apple Inc. Seller and Apple hereby agree as follows:  
1. SERVICES & DELIVERABLES. Seller agrees to perform the services ( "Services" ) and/or provide the  software 
(including all updates, rev isions, error corrections, and subsequent versions thereof, "Software "), materials, 
equipment, hardware,  goods , or deliverables described in a PO (collectively referred to as "Goods" ), in accordance 
with the terms and conditions in this Agreement and the t erms and conditions on the face of the PO, which terms are 
incorpora

In [3]:
# Example usage
print(recursive_summarization(text))

Apple Inc. Purchase Agreement Purchase Order Terms and Conditions This Purchase Agreement (the "Agreement") sets forth the terms and conditions that apply to all purchases of goods and services by Apple from Seller by means of a purchase order (a "PO") issued by Apple to Seller. As used in this Agreement, "Seller" means the entity identified on the face of a PO as "Seller" and its subsidiaries and affiliates, and "Apple" means Apple Inc. Seller and Apple hereby agree as follows: 1. SERVICES & DELIVERABLES. Seller agrees to perform the services ("Services") and/or provide the software (including all updates, revisions, error corrections, and subsequent versions thereof, "Software"), materials, equipment, hardware, goods, or deliverables described in a PO (collectively referred to as "Goods"), in accordance with the terms and conditions in this Agreement and the terms and conditions on the face of the PO, which terms are incorporated herein by reference. Upon acceptance of a PO, shipment